# Create Table Lineage

This notebook provides an example for creating lineage between two tables in Secoda. To get started, you will need to [obtain an API key](https://app.secoda.co/settings/api)

Input your API key and Secoda API endpoint URL below. The URL for the cloud instance is `https://api.secoda.co`. If you are self-hosting Secoda or on the EU instance, you will have to update variable.

In [1]:

import requests

API_KEY = "<api_key>"
SECODA_API_URL = "https://api.secoda.co"

session = requests.Session()
session.headers.update(dict(
    Authorization=f"Bearer {API_KEY}"
))


def build_url(url: str):
    return f"{SECODA_API_URL}{url}"

In [ ]:
def find_secoda_table(title, schema, database, *args, **kwargs):
    res = session.get(
        build_url(f"/table/tables/?title={title}&schema={schema}&database={database}")
    )
    results = res.json().get("results")
    if len(results) >= 1:
        return results[0]
    return None

def set_table_lineage(upstream_table_id, downstream_table_id):
    session.put(
        build_url(f"/lineage/manual/"),
        json=dict(
            from_entity=upstream_table_id,
            to_entity=downstream_table_id,
            direction="DOWNSTREAM",
        )
    )

table_lineage = [
    {
        "upstream_table": {
            "title": "table_1",
            "schema": "schema_1",
            "database": "database_1",
        },
        "downstream_table": {
            "title": "table_2",
            "schema": "schema_2",
            "database": "database_2",
        },
    }
]

for lineage_pair in table_lineage:
    upstream_table = find_secoda_table(
        **lineage_pair["upstream_table"],
    )
    downstream_table = find_secoda_table(
        **lineage_pair["downstream_table"],
    )
    set_table_lineage(upstream_table.get("id", ""), downstream_table.get("id", ""))